In [1]:
import os
from statsmodels.stats.multitest import fdrcorrection
import matplotlib.pyplot as plt

from calc_idr import calc_idr
from manteltest import manteltest
import movieset
import task
from config import get_manual_rating, get_meas_resp_dec, get_pred_resp_dec, RC_PARAMS_DEFAULT
from util import pval_order


%matplotlib inline
plt.rcParams.update(RC_PARAMS_DEFAULT)

In [2]:
nnmodels = ["VGG16_SoundNet", "ResNet50_SoundNet", "ViT_B_16_SoundNet"]
msets = [movieset.WebAdMovieSet, movieset.TVAdMovieSet]
tasks = [
    task.SceneDescriptions,
    task.ImpressionRatings,
    task.AdEffectivenessIndices,
    task.AdPreferenceVotes,
    task.PreferenceRatings,
]

In [3]:
for nnmodel in nnmodels:
    pvals = []
    for m in msets:
        for t in tasks:
            if m not in t.colors: continue
            subjs = t.get_subjects(mset=m)
            pstims_meas = [
                get_meas_resp_dec(mset=m.__name__, task=t.__name__, subj=subj)
                for subj in subjs
            ]
            pstims_pred = [
                get_pred_resp_dec(mset=m.__name__, task=t.__name__, nnmodel=nnmodel, subj=subj)
                for subj in subjs
            ]
            idrval_labels, a_labels, b_labels = calc_idr(pstims_meas, pstims_pred, is_vector=t.is_vector)
            pval_labels = manteltest(a_labels, b_labels)
            pvals.extend(pval_labels)

    _, pvals = fdrcorrection(pvals)
    for m in msets:
        for t in tasks:
            if m not in t.colors: continue
            subjs = t.get_subjects(mset=m)
            pval_labels = pvals[:len(t.item_names)]
            pvals = pvals[len(t.item_names):]
            pstims_meas = [
                get_meas_resp_dec(mset=m.__name__, task=t.__name__, subj=subj)
                for subj in subjs
            ]
            pstims_pred = [
                get_pred_resp_dec(mset=m.__name__, task=t.__name__, nnmodel=nnmodel, subj=subj)
                for subj in subjs
            ]
            idrval_labels, a_labels, b_labels = calc_idr(pstims_meas, pstims_pred, is_vector=t.is_vector)
            for i_label, (idrval, item_name, pval, a, b) in enumerate(zip(idrval_labels, t.item_names, pval_labels, a_labels, b_labels)):
                fig, ax = plt.subplots(figsize=(3, 3))
                ax.set_title(f"Movie set: {m.__name__}\nCategory: {t.__name__}\nLabel: {item_name}")
                ax.scatter(a, b, s=8)
                ax.text(0.02, 0.98, f"IDR = {idrval:.3f}\n{pval_order(pval)}", ha="left", va="top", fontsize=8, transform=ax.transAxes)
                fname_fig = f"./result/{nnmodel}/{m.__name__}/{t.__name__}/idr{i_label:02}.jpg"
                print(f"Saving {fname_fig}")
                os.makedirs(os.path.dirname(fname_fig), exist_ok=True)
                fig.savefig(fname_fig, dpi=200, bbox_inches="tight")
                plt.close(fig)

Saving ./result/VGG16_SoundNet/WebAdMovieSet/SceneDescriptions/idr00.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr00.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr01.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr02.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr03.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr04.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr05.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr06.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr07.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr08.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr09.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr10.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/ImpressionRatings/idr11.jpg
Saving ./result/VGG16_SoundNet/WebAdMovieSet/Impres

## Behavioral IDR

In [4]:
nnmodels = ["VGG16_SoundNet", "ResNet50_SoundNet", "ViT_B_16_SoundNet"]
msets = [movieset.TVAdMovieSet]
tasks = [task.PreferenceRatings]

In [ ]:
for m in msets:
    for t in tasks:
        if m not in t.colors: continue
        pvals = []
        for nnmodel in nnmodels:
            subjs = t.get_subjects(mset=m)
            # NOTE: Referenced Pair dissimilarity matrix was changed into subjective reports
            pstims_meas = [
                get_manual_rating(mset=m.__name__, task=t.__name__, subj=subj)
                for subj in subjs
            ]
            pstims_pred = [
                get_pred_resp_dec(mset=m.__name__, task=t.__name__, nnmodel=nnmodel, subj=subj)
                for subj in subjs
            ]
            idrval_labels, a_labels, b_labels = calc_idr(pstims_meas, pstims_pred, is_vector=t.is_vector)
            pval_labels = manteltest(a_labels, b_labels)
            pvals.extend(pval_labels)

        _, pvals = fdrcorrection(pvals)
        for nnmodel in nnmodels:
            if m not in t.colors: continue
            subjs = t.get_subjects(mset=m)
            pval_labels = pvals[:len(t.item_names)]
            pvals = pvals[len(t.item_names):]
            pstims_meas = [
                get_manual_rating(mset=m.__name__, task=t.__name__, subj=subj)
                for subj in subjs
            ]
            pstims_pred = [
                get_pred_resp_dec(mset=m.__name__, task=t.__name__, nnmodel=nnmodel, subj=subj)
                for subj in subjs
            ]
            idrval_labels, a_labels, b_labels = calc_idr(pstims_meas, pstims_pred, is_vector=t.is_vector)
            for i_label, (idrval, item_name, pval, a, b) in enumerate(zip(idrval_labels, t.item_names, pval_labels, a_labels, b_labels)):
                fig, ax = plt.subplots(figsize=(3, 3))
                ax.set_title(f"Movie set: {m.__name__}\nCategory: {t.__name__}\nLabel: {item_name}")
                ax.scatter(a, b, s=8)
                ax.text(0.02, 0.98, f"IDR = {idrval:.3f}\n{pval_order(pval)}", ha="left", va="top", fontsize=8, transform=ax.transAxes)
                fname_fig = f"./result/{nnmodel}/{m.__name__}/{t.__name__}/behavioral-idr{i_label:02}.jpg"
                print(f"Saving {fname_fig}")
                os.makedirs(os.path.dirname(fname_fig), exist_ok=True)
                fig.savefig(fname_fig, dpi=200, bbox_inches="tight")
                plt.close(fig)

Saving ./result/VGG16_SoundNet/TVAdMovieSet/PreferenceRatings/behavioral-idr00.jpg
Saving ./result/ResNet50_SoundNet/TVAdMovieSet/PreferenceRatings/behavioral-idr00.jpg
Saving ./result/ViT_B_16_SoundNet/TVAdMovieSet/PreferenceRatings/behavioral-idr00.jpg


: 